# PPO Intermedio

Este reto plantea la creación de un flujo de trabajo completo para manejar datos de empleados mediante la implementación de tres clases distintas. El objetivo consiste en separar las responsabilidades de validación de datos, análisis estadístico y generación de reportes.

Se requiere contar con una lista de diccionarios llamada `empleados`, donde cada diccionario represente a un empleado con las siguientes claves obligatorias:

In [24]:
empleado = {
    "id": str,                    # Identificador único del empleado (UUID)
    "nombre": str,                # Nombre completo del empleado
    "edad": int,                  # Edad del empleado
    "genero": str,                # Género (Masculino, Femenino, etc.)
    "departamento": str,          # Departamento al que pertenece
    "cargo": str,                 # Cargo que desempeña en la empresa
    "email": str,                 # Correo electrónico
    "telefono": str,              # Número de teléfono
    "direccion": str,             # Dirección física
    "salario": float,             # Salario base anual (o mensual, según convenga)
    "fecha_contratacion": str,    # Fecha de contratación en formato YYYY-MM-DD
    "desempeno": float,           # Valor numérico que represente el desempeño (ej. 0 a 10)
    "activo": bool                # Indica si el empleado sigue activo en la empresa
}

In [25]:
import json
import re
import datetime
from fpdf import FPDF
import logging

In [26]:
class Logger:
    def __init__(self, log_file="app.log"):
        # Crear logger
        self.logger = logging.getLogger("App_Logger")
        self.logger.setLevel(logging.DEBUG)

        # Evitar duplicación de handlers
        if not self.logger.handlers:
            # Formato del log
            formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

            # Crear manejador para archivo
            file_handler = logging.FileHandler(log_file, encoding = "utf-8")
            file_handler.setFormatter(formatter)
            file_handler.setLevel(logging.DEBUG)

            # Crear manejador para imprimir consola
            console_handler = logging.StreamHandler()
            console_handler.setFormatter(formatter)
            console_handler.setLevel(logging.INFO)

            # Agregar handlers al logger
            self.logger.addHandler(file_handler)
            self.logger.addHandler(console_handler)

    def get_logger(self):
        return self.logger

In [27]:
class ReadData:
    def __init__(self, filename, logger):
        self.filename = filename
        self.logger = logger
        
    def read_json(self):
        #leer el archivo json de manera correcta
        with open(self.filename, "r", encoding="utf-8") as f:
            employees = [json.loads(line) for line in f] 
            self.logger.info(f"Datos obtenidos de {self.filename}")
        return employees


In [28]:
class EmployeeDataLoader:
    def __init__(self, employees, logger):
        self.employees = employees
        self.logger = logger
        
    def get_clean_data(self):
                
        mandatory_keys = {
            "id": str,                    # Identificador único del empleado (UUID)
            "nombre": str,                # Nombre completo del empleado
            "edad": int,                  # Edad del empleado
            "genero": str,                # Género (Masculino, Femenino, etc.)
            "departamento": str,          # Departamento al que pertenece
            "cargo": str,                 # Cargo que desempeña en la empresa
            "email": str,                 # Correo electrónico
            "telefono": str,              # Número de teléfono
            "direccion": str,             # Dirección física
            "salario": float,             # Salario base anual (o mensual, según convenga)
            "fecha_contratacion": str,    # Fecha de contratación en formato YYYY-MM-DD
            "desempeno": float,           # Valor numérico que represente el desempeño (ej. 0 a 10)
            "activo": bool                # Indica si el empleado sigue activo en la empresa
        }
        
        activo_dict = {
            "activo": True, "on": True, "1": True, "yes": True, "y": True, 1: True, "true": True, "t": True,
            "inactivo": False, "off": False, "0": False, "no": False, "n": False, 0: False, "false": False, "f": False
                      }
        
        valid_employees = []
        errors = []
        
        for i, dic in enumerate(self.employees):
            #verificar la existencia de claves:
            missing = [clave for clave in mandatory_keys.keys()
                         if clave not in dic]
            if missing:
                errors.append(f"Error en empleado {i}: faltan las claves: {missing}")
                self.logger.warning(f"Error en empleado {i}: faltan las claves: {missing}")
                continue
            
            #cambiar a boleano
            dic["activo"] = str(dic["activo"]).lower().strip()
            dic["activo"] = activo_dict.get(str(dic["activo"]).strip(), None)
            
            #ambiar a int/float
            dic["edad"] = int(dic["edad"])
            dic["salario"] = float(dic["salario"])
            dic["desempeno"] = float(dic["desempeno"])
            
            #comprobar fechas
            if dic.get("fecha_contratacion"):
                if isinstance(dic["fecha_contratacion"], str):
                    if not re.match(r"^\d{4}-\d{2}-\d{2}$", dic["fecha_contratacion"]):
                        errors.append(f"Error en empleado {i}: la fecha de contratación no es válida")
                        self.logger.warning(f"Error en empleado {i}: faltan las claves: {missing}")
                        continue
                
            empleado_valido = True
            for clave, tipo in mandatory_keys.items(): 
                if not isinstance(dic[clave], tipo):
                    errors.append(f"Error en empleado {i}: la clave '{clave}' no es del tipo {tipo.__name__}")
                    self.logger.warning(f"Error en empleado {i}: la clave '{clave}' no es del tipo {tipo.__name__}")
                    empleado_valido = False 
            if empleado_valido:
                valid_employees.append(dic)
        self.logger.info(f"Se creo la lista de empleados validos")
        return valid_employees, errors

In [29]:
class EmployerAnalyzer:
    def __init__(self, valid_employees, logger):
        self.valid_employees = valid_employees
        self.logger = logger
        self.total = len(valid_employees)
        
        #Cambiar el tipo de dato de fecha_contratacion a datetime.date
        for employee in self.valid_employees:
            if "fecha_contratacion" in employee and isinstance(employee["fecha_contratacion"], str):
                try:
                    employee["fecha_contratacion"] = datetime.datetime.strptime(employee["fecha_contratacion"], "%Y-%m-%d").date()
                except ValueError:
                    employee["fecha_contratacion"] = None
        self.logger.info(f"Se cambio el tipo de dato 'fecha_contratacion' de str a datetime")
        
    def get_average_salary(self):
        #calcular el salario promedio de los empleados
        if not self.valid_employees:
            self.logger.warning(f"No hay datos validos")
        else:
            average_salary = sum([employee["salario"] for employee in self.valid_employees]) / self.total
            self.logger.info(f"Se aplico el metodo get_average_salary")
            return round(average_salary, 2)
        
    def count_by_apartment(self):
        #contar la cantidad de empleados por departamento
        department_counter = {}
        for empleado in self.valid_employees:
            departament = empleado["departamento"]
            if departament not in department_counter:
                department_counter[departament] = 0
            department_counter[departament] += 1
        self.logger.info(f"Se aplico el metodo count_by_department")
        return department_counter
    
    def average_performance(self):
        #calcular el desempeño promedio de los empleados 
        performance_average = sum([employee["desempeno"] for employee in self.valid_employees]) / self.total
        self.logger.info(f"Se aplico el metodo average_performance")
        return round(performance_average, 2)
    
    def percentage_of_active_employees(self):
        #calcular el porcentaje de empleados activos
        
       active_employees = sum(1 for employee in self.valid_employees if employee["activo"] == True)
       percent_active_employees = (active_employees / self.total) * 100
       self.logger.info(f"Se aplico el metodo percentage_of_active_employees")
       return round(percent_active_employees, 2)
    
    def average_days_employed(self):
        #calcular el promedio de dias trabajados en la empresa
                    
       today= datetime.date.today()
       total_days = sum((today- employee["fecha_contratacion"]).days
                          for employee in self.valid_employees
                          if employee["fecha_contratacion"] is not None)
       average_days = total_days / self.total
       self.logger.info(f"Se aplico el metodo average_days_employeed")
       return round(average_days, 2)

In [30]:
#crear el logger
logger = Logger().get_logger()

In [31]:
#Abrir el archivo Json
data_file = ReadData("empleados.json", logger)
employee = data_file.read_json()

2025-04-04 20:24:54,526 - App_Logger - INFO - Datos obtenidos de empleados.json


In [32]:
# Validar los datos
data_loader = EmployeeDataLoader(employee, logger)
datos_limpios, errores = data_loader.get_clean_data()

2025-04-04 20:24:54,555 - App_Logger - WARNING - Error en empleado 11: faltan las claves: ['fecha_contratacion']
2025-04-04 20:24:54,557 - App_Logger - WARNING - Error en empleado 12: faltan las claves: ['fecha_contratacion']


2025-04-04 20:24:54,558 - App_Logger - WARNING - Error en empleado 21: faltan las claves: ['telefono']
2025-04-04 20:24:54,559 - App_Logger - WARNING - Error en empleado 30: faltan las claves: ['edad']
2025-04-04 20:24:54,561 - App_Logger - WARNING - Error en empleado 35: faltan las claves: ['fecha_contratacion']
2025-04-04 20:24:54,562 - App_Logger - WARNING - Error en empleado 41: faltan las claves: ['edad']
2025-04-04 20:24:54,563 - App_Logger - WARNING - Error en empleado 48: faltan las claves: ['fecha_contratacion']
2025-04-04 20:24:54,564 - App_Logger - WARNING - Error en empleado 51: faltan las claves: ['telefono']
2025-04-04 20:24:54,565 - App_Logger - WARNING - Error en empleado 66: faltan las claves: ['fecha_contratacion']
2025-04-04 20:24:54,566 - App_Logger - WARNING - Error en empleado 75: faltan las claves: ['fecha_contratacion']
2025-04-04 20:24:54,567 - App_Logger - WARNING - Error en empleado 91: faltan las claves: ['fecha_contratacion']
2025-04-04 20:24:54,568 - App_L

In [33]:
class EmployeeReportGenerator(EmployerAnalyzer):
    def __init__(self, valid_employees, logger):
        super().__init__(valid_employees, logger)
        self.logger = logger
        
    def generate_text_report(self, filename = "reporte.pdf"):
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=12)
        
        pdf.set_font("Arial", 'B', 16)
        pdf.cell(200, 10, txt="Reporte de Empleados", ln=True, align='C')
        pdf.ln(10)
        self.logger.info(f"Se aplicaron valores de creacion de reporte en PDF")
        
        pdf.set_font("Arial", size=12)
        pdf.cell(200, 10, txt=f"Total de empleados: {self.total}", ln = True)
        pdf.cell(200, 10, txt=f"Salario promedio: ${self.get_average_salary()}", ln = True)
        pdf.cell(200, 10, txt=f"Desempeño promedio: {self.average_performance()}", ln = True)
        pdf.cell(200, 10, txt=f"Porcentaje de empleados activos: {self.percentage_of_active_employees()}%", ln = True)
        pdf.cell(200, 10, txt=f"Promedio de días trabajados: {self.average_days_employed()} días", ln = True)
        
        pdf.ln(10)
        pdf.set_font("Arial", 'B', 12)
        pdf.cell(200, 10, txt="\nCantidad de empleados por departamento:", ln = True)
        pdf.set_font("Arial", size=12)
            
        for department, quantity in self.count_by_apartment().items():
            pdf.cell(200, 10, txt=f"{department}: {quantity} empleados", ln = True)
        pdf.output(filename)
        self.logger.info(f"Se aplico el metodo generate_text_report")
        return filename

In [34]:
report_generator = EmployeeReportGenerator(datos_limpios, logger)

    # Generar y mostrar el reporte
report = report_generator.generate_text_report()
print(report)

2025-04-04 20:25:09,717 - App_Logger - INFO - Se cambio el tipo de dato 'fecha_contratacion' de str a datetime
2025-04-04 20:25:09,720 - App_Logger - INFO - Se aplicaron valores de creacion de reporte en PDF
2025-04-04 20:25:09,754 - App_Logger - INFO - Se aplico el metodo get_average_salary
2025-04-04 20:25:09,784 - App_Logger - INFO - Se aplico el metodo average_performance
2025-04-04 20:25:09,820 - App_Logger - INFO - Se aplico el metodo percentage_of_active_employees
2025-04-04 20:25:09,863 - App_Logger - INFO - Se aplico el metodo average_days_employeed
2025-04-04 20:25:09,914 - App_Logger - INFO - Se aplico el metodo count_by_department
2025-04-04 20:25:09,917 - App_Logger - INFO - Se aplico el metodo generate_text_report


reporte.pdf


## **Requisitos:**


1. **Primera clase: EmployeeDataLoader**  
   - **Constructor**  
     - Debe recibir la lista `empleados` y guardarla en un atributo de instancia.
   - **Método: get_clean_data()**  
     - Se debe validar que cada diccionario contenga las **trece** claves indicadas (`id`, `nombre`, `edad`, `genero`, `departamento`, `cargo`, `email`, `telefono`, `direccion`, `salario`, `fecha_contratacion`, `desempeno` y `activo`).
     - Se debe asegurar que `edad` sea entero, `salario` sea flotante, `activo` sea booleano, etc. Si algún valor no cumple con las expectativas, se cataloga como “dato sucio”.
     - Debe retornar una nueva lista con **solo** los diccionarios válidos y manejar los inválidos de alguna forma.

2. **Segunda clase: EmployeeAnalyzer**  
   - **Constructor**  
     - Debe recibir la lista resultante de `get_clean_data()` (diccionarios validados).
   - **Métodos**  
     - `get_average_salary()`:  
       - Calcula el promedio de `salario`.
     - `count_by_department()`:  
       - Retorna cuántos empleados hay en cada `departamento`.
     - `average_performance()`:  
       - Calcula el promedio de `desempeno`.
     - `percentage_of_active_employees()`:  
       - Retorna el porcentaje de empleados con `activo == True`.
     - `average_days_employed()`:
       - Calcula el promedio de los días empleados para los empleados activos.

3. **Tercera clase (con herencia): EmployeeReportGenerator**  
- Esta clase debe **heredar** de `EmployeeAnalyzer`. De esta forma, tendrá acceso directo a los métodos de análisis.
  
- **Constructor**  
  - Debe recibir la lista validada y enviarla al constructor de la clase padre (`EmployeeAnalyzer`) mediante `super().__init__()`.
  
- **Método: generate_text_report()**  
  - Debe generar un reporte (en **PDF**) que compile la información calculada por todos los métodos de `EmployeeAnalyzer`.

## **Estructura de archivos propuesta**

```
my_project/
├─ main.py
├─ data_loader.py
├─ analyzer.py
└─ report_generator.py
```

- **data_loader.py**  
  Contiene la clase `EmployeeDataLoader`.
- **analyzer.py**  
  Contiene la clase `EmployeeAnalyzer`.
- **report_generator.py**  
  Contiene la clase `EmployeeReportGenerator`, que hereda de `EmployeeAnalyzer`.
- **main.py**  
  Actúa como punto de entrada, define la lista `empleados`, instancia las clases y ejecuta el flujo general.

## **Pseudo código sugerido**

#### `main.py`

``` python
from data_loader import EmployeeDataLoader
from report_generator import EmployeeReportGenerator

def main():
    # Cragar los datos
    # AQUÍ SE PUEDE CREAR OTRA CLASE!!!

    # Instanciar el cargador y obtener datos limpios
    loader = EmployeeDataLoader(empleados)
    clean_data = loader.get_clean_data()

    # Instanciar la clase que hereda de EmployeeAnalyzer (EmployeeReportGenerator)
    report_generator = EmployeeReportGenerator(clean_data)

    # Generar y mostrar el reporte
    report = report_generator.generate_text_report()
    print(report)

if __name__ == "__main__":
    main()
```
<br>

#### `employee_data_loader.py`

``` python

class EmployeeDataLoader:
    """
    Clase encargada de recibir y validar la lista de empleados.
    Verifica que cada diccionario contenga todos los campos requeridos
    y que dichos campos sean del tipo esperado.
    """

    def __init__(self, employees):
```
<br>


### `employee_analyzer.py`

``` python

class EmployeeAnalyzer:
    """
    Clase enfocada en realizar cálculos y estadísticas 
    a partir de datos de empleados limpios y validados.
    """

    def __init__(self, employees):
        """
        Constructor que recibe la lista de empleados ya validada.
        """
        self.employees = employees
```
<br>


### `employee_report_generator.py`

``` python

from employee_analyzer import EmployeeAnalyzer

class EmployeeReportGenerator(EmployeeAnalyzer):
    """
    Clase que hereda de EmployeeAnalyzer.
    Agrega la responsabilidad de generar un reporte textual 
    basado en los cálculos y estadísticas disponibles.
    """

    def __init__(self, employees):
        super().__init__(employees)
```
